In [21]:
import escher
from escher import Builder
import cobra
from time import sleep
import os
import pickle

In [22]:
# Load the model into cobra
# Relative path to model file saved locally on Helen's computer, but no changes to the model since downloading it from BiGG
model = cobra.io.read_sbml_model('../../../../GEM-repos/mit1002-model/model.xml')

'' is not a valid SBML 'SId'.


In [23]:
# Load the solutions
# Load the results from the COBRA simulations
with open('results.pkl', 'rb') as f:
    cobra_results = pickle.load(f)

glc_only_fba = cobra_results[0]
glc_only_pfba = cobra_results[1]
ace_only_fba = cobra_results[2]
ace_only_pfba = cobra_results[3]
mix_fba = cobra_results[4]
mix_pfba = cobra_results[5]

In [24]:
# Make a map with the solution for glucose only
glc_only_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = glc_only_fba.fluxes
)

In [25]:
# View the map
glc_only_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -3.5239759075541848, 'rxn02342_c0': 0.0, 'rxn06493_c…

In [26]:
# Make a map with the solution for acetate only
ace_only_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = ace_only_fba.fluxes
)

In [27]:
ace_only_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -0.7524648226980366, 'rxn02342_c0': 0.0, 'rxn06493_c…

In [28]:
glc_only_fba

,fluxes,reduced_costs
rxn00247_c0,0.000000,-6.368166e-03
rxn05625_c0,-3.523976,-2.649918e-17
rxn02342_c0,0.000000,-8.756229e-02
rxn06493_c0,0.025807,1.169186e-16
rxn02376_c0,0.000000,-1.133212e-18
...,...,...
EX_cpd00033_e0,0.000000,-2.220898e-01
EX_cpd00060_e0,0.000000,-3.128362e-01
DM_cpd15302_c0,0.000000,0.000000e+00
DM_cpd02701_c0,0.003099,0.000000e+00


In [29]:
# Make a map with the solution for glucose only using pFBA
glc_only_pfba_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = glc_only_pfba.fluxes
)

In [30]:
glc_only_pfba_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -3.523975907554188, 'rxn02342_c0': 0.0, 'rxn06493_c0…

In [31]:
# Make a map with the solution for acetate only using pFBA
ace_only_pfba_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = ace_only_pfba.fluxes
)

In [32]:
ace_only_pfba_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -0.7524648226980615, 'rxn02342_c0': 0.0, 'rxn06493_c…